## Order a set of datasets with the dataset popularity listing

This is an attempt to use the dataset popularity listing to select the most relevant samples among a list of MC samples proposed as legacy.

In [1]:
import pandas as pd
import numpy as np


### Read in the dataset listings

#### Dataset listing from CMSDAS

The dataset listing is from CMSDAS (https://cmsweb.cern.ch/das/request?view=plain&limit=50&instance=prod%2Fglobal&input=dataset%3D%2F*%2F*START42*%2FAODSIM)

Read it in to a dataframe.

In [2]:
datasets = pd.read_csv('datasets.txt', sep=" ", header=None)

In [3]:
datasets.head()

,0
0,/2B1Jet_TuneZ2_7TeV-alpgen-pythia6/Fall11-PU_S...
1,/2B2C1Jet_TuneZ2_7TeV-alpgen-pythia6/Fall11-PU...
2,/2B2C_TuneZ2_7TeV-alpgen-pythia6/Fall11-PU_S6_...
3,/2B2Jets_TuneZ2_7TeV-alpgen-pythia6/Fall11-PU_...
4,/2B3Jets_TuneZ2_7TeV-alpgen-pythia6/Fall11-PU_...


#### Dataset popularity list

The dataset popularity json file is from https://cmsweb.cern.ch/popdb/popularity/dataSetTable

Read it in to a dataframe.

In [4]:
datapop = pd.read_json('datasets_start2010-01-01_stop2012-03-31.json')

In [5]:
datapop.head()


,DATA,SITENAME
0,"{'NACC': 2900135, 'TOTCPU': 3296253, 'NUSERS':...",summary
1,"{'NACC': 2026464, 'TOTCPU': 2056789, 'NUSERS':...",summary
2,"{'NACC': 1026045, 'TOTCPU': 2053499, 'NUSERS':...",summary
3,"{'NACC': 1262370, 'TOTCPU': 1915483, 'NUSERS':...",summary
4,"{'NACC': 1537482, 'TOTCPU': 1765946, 'NUSERS':...",summary


It comes in two parts, DATA and SITENAME, but only the DATA part is relevant in this context:

In [6]:
datapop['DATA']

0        {'NACC': 2900135, 'TOTCPU': 3296253, 'NUSERS':...
1        {'NACC': 2026464, 'TOTCPU': 2056789, 'NUSERS':...
2        {'NACC': 1026045, 'TOTCPU': 2053499, 'NUSERS':...
3        {'NACC': 1262370, 'TOTCPU': 1915483, 'NUSERS':...
4        {'NACC': 1537482, 'TOTCPU': 1765946, 'NUSERS':...
5        {'NACC': 987978, 'TOTCPU': 1167262, 'NUSERS': ...
6        {'NACC': 923959, 'TOTCPU': 1059471, 'NUSERS': ...
7        {'NACC': 2851106, 'TOTCPU': 966514, 'NUSERS': ...
8        {'NACC': 500687, 'TOTCPU': 922797, 'NUSERS': 1...
9        {'NACC': 15017490, 'TOTCPU': 900783, 'NUSERS':...
10       {'NACC': 688387, 'TOTCPU': 861974, 'NUSERS': 8...
11       {'NACC': 657485, 'TOTCPU': 746178, 'NUSERS': 1...
12       {'NACC': 329100, 'TOTCPU': 712978, 'NUSERS': 6...
13       {'NACC': 391693, 'TOTCPU': 689265, 'NUSERS': 8...
14       {'NACC': 284142, 'TOTCPU': 634980, 'NUSERS': 1...
15       {'NACC': 437517, 'TOTCPU': 567978, 'NUSERS': 1...
16       {'NACC': 243920, 'TOTCPU': 556346, 'NUSERS': 2.

In [7]:
datapop['DATA'][0]


{'COLLNAME': '/WJetsToLNu_TuneZ2_7TeV-madgraph-tauola/Summer11-PU_S4_START42_V11-v1/AODSIM',
 'NACC': 2900135,
 'NUSERS': 3667,
 'RNACC': '2.7',
 'RNUSERS': '1.3',
 'RTOTCPU': '5.2',
 'TOTCPU': 3296253}

`datapop` is a dataframe, but not in the format which is needed for data operations. Single elements in DATA contain the information we need, but we want the keys NACC, NUSERS etc in the column header.

To get a dataframe in a useful format,
see https://stackoverflow.com/questions/29681906/python-pandas-dataframe-from-series-of-dict

So we start from a dataframe:

In [8]:
type(datapop)

pandas.core.frame.DataFrame

DATA part in the dataframe is a series:

In [9]:
type(datapop['DATA'])

pandas.core.series.Series

Each element in DATA is a dict:

In [10]:
type(datapop["DATA"].ix[0])

dict

We want the keys of this dict as column headers:

In [11]:
datapop["DATA"].ix[0].keys()

dict_keys(['NACC', 'TOTCPU', 'NUSERS', 'COLLNAME', 'RNACC', 'RNUSERS', 'RTOTCPU'])

This command from the instructions above does the magic (get a dataframe from a series of dicts...) :

In [12]:
new_df = pd.DataFrame(list(datapop['DATA']))

In [13]:
new_df.head()

,COLLNAME,NACC,NUSERS,RNACC,RNUSERS,RTOTCPU,TOTCPU
0,/WJetsToLNu_TuneZ2_7TeV-madgraph-tauola/Summer...,2900135,3667,2.7,1.3,5.2,3296253
1,/DYJetsToLL_TuneZ2_M-50_7TeV-madgraph-tauola/S...,2026464,3786,1.9,1.3,3.3,2056789
2,/TTJets_TuneZ2_7TeV-madgraph-tauola/Fall11-PU_...,1026045,948,0.9,0.3,3.2,2053499
3,/SingleMu/Run2011B-PromptReco-v1/AOD,1262370,2238,1.2,0.8,3.0,1915483
4,/SingleMu/Run2011A-PromptReco-v4/AOD,1537482,3571,1.4,1.3,2.8,1765946


Sort the dataframe for the number of users:

In [14]:
new_df = new_df.sort_values('NUSERS',ascending=False)

### Find out the popularity order for a selected list of datasets

Find in this ordered popularity list the dataset names which appear in `datasets`... In the case of a known dataset name, the corresponding row in the dataframe can be found with:

In [15]:
new_df.loc[new_df['COLLNAME'] == "/DY1JetToLL_M-10To50_TuneZ2_7TeV-madgraph/Fall11-PU_S6_START42_V14B-v1/AODSIM"]

,COLLNAME,NACC,NUSERS,RNACC,RNUSERS,RTOTCPU,TOTCPU
1728,/DY1JetToLL_M-10To50_TuneZ2_7TeV-madgraph/Fall...,932,9,0.0,0.0,0.0,765


Instead of using the exact string for the search, we want to take it from the datasets listing. Just for an exercise, we can find this string from the dataset listing, and then use its location in the search:

In [16]:
datasets.loc[datasets[0] == "/DY1JetToLL_M-10To50_TuneZ2_7TeV-madgraph/Fall11-PU_S6_START42_V14B-v1/AODSIM"]

,0
154,/DY1JetToLL_M-10To50_TuneZ2_7TeV-madgraph/Fall...


In [17]:
new_df.loc[new_df['COLLNAME'] == datasets[0][154]]

,COLLNAME,NACC,NUSERS,RNACC,RNUSERS,RTOTCPU,TOTCPU
1728,/DY1JetToLL_M-10To50_TuneZ2_7TeV-madgraph/Fall...,932,9,0.0,0.0,0.0,765


Now instead of looking for the rows one by one, we can find all corresponding rows in the dataframe.
Write the list of datasets from the `datasets` to a list, see https://stackoverflow.com/questions/19585280/convert-a-row-in-pandas-into-list

In [18]:
datalist = datasets[0].values.tolist()

Select rows based on a list, see https://stackoverflow.com/questions/12096252/use-a-list-of-values-to-select-rows-from-a-pandas-dataframe?rq=1

In [19]:
new_df[new_df['COLLNAME'].isin(datalist)]

,COLLNAME,NACC,NUSERS,RNACC,RNUSERS,RTOTCPU,TOTCPU
6,/DYJetsToLL_TuneZ2_M-50_7TeV-madgraph-tauola/F...,923959,1266,0.9,0.4,1.7,1059471
5,/WJetsToLNu_TuneZ2_7TeV-madgraph-tauola/Fall11...,987978,1180,0.9,0.4,1.8,1167262
2,/TTJets_TuneZ2_7TeV-madgraph-tauola/Fall11-PU_...,1026045,948,0.9,0.3,3.2,2053499
163,/TTJets_TuneZ2_7TeV-madgraph-tauola/Fall11-PU_...,74208,397,0.1,0.1,0.1,69849
50,/DYToMuMu_M-20_CT10_TuneZ2_7TeV-powheg-pythia/...,214307,393,0.2,0.1,0.4,249990
61,/DYToEE_M-20_CT10_TuneZ2_7TeV-powheg-pythia/Fa...,173031,266,0.2,0.1,0.3,194591
213,/WZ_TuneZ2_7TeV_pythia6_tauola/Fall11-PU_S6_ST...,48271,260,0.0,0.1,0.1,51386
43,/QCD_Pt-30to80_EMEnriched_TuneZ2_7TeV-pythia/F...,368661,252,0.3,0.1,0.5,288726
210,/ZZ_TuneZ2_7TeV_pythia6_tauola/Fall11-PU_S6_ST...,48289,250,0.0,0.1,0.1,52181
204,/WW_TuneZ2_7TeV_pythia6_tauola/Fall11-PU_S6_ST...,37441,233,0.0,0.1,0.1,54748


Write the ordered list in a text file

In [20]:
#new_df[new_df['COLLNAME'].isin(datalist)]['COLLNAME'].values

In [21]:
np.savetxt(r'np.txt', new_df[new_df['COLLNAME'].isin(datalist)]['COLLNAME'].values, fmt='%s')

### Popularity listing by process name

Try to see whether searching for just the first part of the dataset name makes a difference. To do this, one needs first to combine the different reprocessings in the popularity listing in one entry and sum the value for NACC, NUSERS and TOTCPU. Use only the first part of the dataset name (between the two first "/"). Then for search, use the corresponding part of the dataset name in the listing.

Split the string first, see https://stackoverflow.com/questions/32425334/splitting-a-string-in-a-python-dataframe

In [22]:
new_df.COLLNAME.str.split('/')


1        [, DYJetsToLL_TuneZ2_M-50_7TeV-madgraph-tauola...
0        [, WJetsToLNu_TuneZ2_7TeV-madgraph-tauola, Sum...
4                [, SingleMu, Run2011A-PromptReco-v4, AOD]
7               [, SingleMu, Run2011A-May10ReReco-v1, AOD]
3                [, SingleMu, Run2011B-PromptReco-v1, AOD]
18               [, DoubleMu, Run2011A-PromptReco-v4, AOD]
11         [, DoubleElectron, Run2011A-PromptReco-v4, AOD]
23       [, TTJets_TuneZ2_7TeV-madgraph-tauola, Summer1...
40              [, DoubleMu, Run2011A-May10ReReco-v1, AOD]
32        [, DoubleElectron, Run2011A-May10ReReco-v1, AOD]
15               [, DoubleMu, Run2011B-PromptReco-v1, AOD]
17               [, SingleMu, Run2011A-PromptReco-v6, AOD]
30         [, SingleElectron, Run2011A-PromptReco-v4, AOD]
6        [, DYJetsToLL_TuneZ2_M-50_7TeV-madgraph-tauola...
41                [, SingleMu, Run2011A-05Aug2011-v1, AOD]
28                 [, Photon, Run2011A-PromptReco-v4, AOD]
8                      [, HT, Run2011A-PromptReco-v4, AO

Insert the split parts of the full dataset name (process, reprocessing and datatype) to the dataframe:

In [23]:
new_df['process'] = new_df.COLLNAME.str.split('/').str[1]
new_df['reprocessing'] = new_df.COLLNAME.str.split('/').str[2]
new_df['datatype'] = new_df.COLLNAME.str.split('/').str[3]

In [24]:
new_df.head()

,COLLNAME,NACC,NUSERS,RNACC,RNUSERS,RTOTCPU,TOTCPU,process,reprocessing,datatype
1,/DYJetsToLL_TuneZ2_M-50_7TeV-madgraph-tauola/S...,2026464,3786,1.9,1.3,3.3,2056789,DYJetsToLL_TuneZ2_M-50_7TeV-madgraph-tauola,Summer11-PU_S4_START42_V11-v1,AODSIM
0,/WJetsToLNu_TuneZ2_7TeV-madgraph-tauola/Summer...,2900135,3667,2.7,1.3,5.2,3296253,WJetsToLNu_TuneZ2_7TeV-madgraph-tauola,Summer11-PU_S4_START42_V11-v1,AODSIM
4,/SingleMu/Run2011A-PromptReco-v4/AOD,1537482,3571,1.4,1.3,2.8,1765946,SingleMu,Run2011A-PromptReco-v4,AOD
7,/SingleMu/Run2011A-May10ReReco-v1/AOD,2851106,3082,2.6,1.1,1.5,966514,SingleMu,Run2011A-May10ReReco-v1,AOD
3,/SingleMu/Run2011B-PromptReco-v1/AOD,1262370,2238,1.2,0.8,3.0,1915483,SingleMu,Run2011B-PromptReco-v1,AOD


Separate the listing for MC and data - as a first approximation data does not have - or _ in the process name.

In [25]:
#new_df = new_df[(new_df['datatype'] != 'AOD')] 
#new_df = new_df[(new_df['datatype'] != 'RECO')]
#new_df = new_df[(new_df['datatype'] != 'RAW')]
mc_df  = new_df[new_df["process"].str.contains('-|_')==True]
data_df = new_df[new_df["process"].str.contains('-|_')==False]

mc_df.head(20)

,COLLNAME,NACC,NUSERS,RNACC,RNUSERS,RTOTCPU,TOTCPU,process,reprocessing,datatype
1,/DYJetsToLL_TuneZ2_M-50_7TeV-madgraph-tauola/S...,2026464,3786,1.9,1.3,3.3,2056789,DYJetsToLL_TuneZ2_M-50_7TeV-madgraph-tauola,Summer11-PU_S4_START42_V11-v1,AODSIM
0,/WJetsToLNu_TuneZ2_7TeV-madgraph-tauola/Summer...,2900135,3667,2.7,1.3,5.2,3296253,WJetsToLNu_TuneZ2_7TeV-madgraph-tauola,Summer11-PU_S4_START42_V11-v1,AODSIM
23,/TTJets_TuneZ2_7TeV-madgraph-tauola/Summer11-P...,329509,1898,0.3,0.7,0.7,434610,TTJets_TuneZ2_7TeV-madgraph-tauola,Summer11-PU_S4_START42_V11-v1,AODSIM
6,/DYJetsToLL_TuneZ2_M-50_7TeV-madgraph-tauola/F...,923959,1266,0.9,0.4,1.7,1059471,DYJetsToLL_TuneZ2_M-50_7TeV-madgraph-tauola,Fall11-PU_S6_START42_V14B-v1,AODSIM
5,/WJetsToLNu_TuneZ2_7TeV-madgraph-tauola/Fall11...,987978,1180,0.9,0.4,1.8,1167262,WJetsToLNu_TuneZ2_7TeV-madgraph-tauola,Fall11-PU_S6_START42_V14B-v1,AODSIM
27,/WJetsToLNu_TuneZ2_7TeV-madgraph-tauola/Spring...,573031,1076,0.5,0.4,0.6,401126,WJetsToLNu_TuneZ2_7TeV-madgraph-tauola,Spring11-PU_S1_START311_V1G1-v1,AODSIM
2,/TTJets_TuneZ2_7TeV-madgraph-tauola/Fall11-PU_...,1026045,948,0.9,0.3,3.2,2053499,TTJets_TuneZ2_7TeV-madgraph-tauola,Fall11-PU_S6_START42_V14B-v2,AODSIM
10,/QCD_Pt-30to80_EMEnriched_TuneZ2_7TeV-pythia/S...,688387,803,0.6,0.3,1.4,861974,QCD_Pt-30to80_EMEnriched_TuneZ2_7TeV-pythia,Summer11-PU_S4_START42_V11-v1,AODSIM
101,/T_TuneZ2_t-channel_7TeV-powheg-tauola/Summer1...,104410,731,0.1,0.3,0.2,111721,T_TuneZ2_t-channel_7TeV-powheg-tauola,Summer11-PU_S4_START42_V11-v1,AODSIM
46,/DYToMuMu_M-20_CT10_TuneZ2_7TeV-powheg-pythia/...,440509,724,0.4,0.3,0.4,279291,DYToMuMu_M-20_CT10_TuneZ2_7TeV-powheg-pythia,Summer11-PU_S4_START42_V11-v1,AODSIM


In [26]:
data_df.head()

,COLLNAME,NACC,NUSERS,RNACC,RNUSERS,RTOTCPU,TOTCPU,process,reprocessing,datatype
4,/SingleMu/Run2011A-PromptReco-v4/AOD,1537482,3571,1.4,1.3,2.8,1765946,SingleMu,Run2011A-PromptReco-v4,AOD
7,/SingleMu/Run2011A-May10ReReco-v1/AOD,2851106,3082,2.6,1.1,1.5,966514,SingleMu,Run2011A-May10ReReco-v1,AOD
3,/SingleMu/Run2011B-PromptReco-v1/AOD,1262370,2238,1.2,0.8,3.0,1915483,SingleMu,Run2011B-PromptReco-v1,AOD
18,/DoubleMu/Run2011A-PromptReco-v4/AOD,502407,1959,0.5,0.7,0.8,511015,DoubleMu,Run2011A-PromptReco-v4,AOD
11,/DoubleElectron/Run2011A-PromptReco-v4/AOD,657485,1942,0.6,0.7,1.2,746178,DoubleElectron,Run2011A-PromptReco-v4,AOD


Check which rows have the same process:

In [27]:
mc_df.loc[mc_df['process'] == "WJetsToLNu_TuneZ2_7TeV-madgraph-tauola"]

,COLLNAME,NACC,NUSERS,RNACC,RNUSERS,RTOTCPU,TOTCPU,process,reprocessing,datatype
0,/WJetsToLNu_TuneZ2_7TeV-madgraph-tauola/Summer...,2900135,3667,2.7,1.3,5.2,3296253,WJetsToLNu_TuneZ2_7TeV-madgraph-tauola,Summer11-PU_S4_START42_V11-v1,AODSIM
5,/WJetsToLNu_TuneZ2_7TeV-madgraph-tauola/Fall11...,987978,1180,0.9,0.4,1.8,1167262,WJetsToLNu_TuneZ2_7TeV-madgraph-tauola,Fall11-PU_S6_START42_V14B-v1,AODSIM
27,/WJetsToLNu_TuneZ2_7TeV-madgraph-tauola/Spring...,573031,1076,0.5,0.4,0.6,401126,WJetsToLNu_TuneZ2_7TeV-madgraph-tauola,Spring11-PU_S1_START311_V1G1-v1,AODSIM
1449,/WJetsToLNu_TuneZ2_7TeV-madgraph-tauola/Summer...,328937,111,0.3,0.0,0.0,1104,WJetsToLNu_TuneZ2_7TeV-madgraph-tauola,Summer11-PU_S4_START42_V11-v1,GEN-SIM-RECO
138,/WJetsToLNu_TuneZ2_7TeV-madgraph-tauola/Fall11...,168087,105,0.2,0.0,0.1,84198,WJetsToLNu_TuneZ2_7TeV-madgraph-tauola,Fall11-PU_S6-START44_V5-v1,AODSIM
887,/WJetsToLNu_TuneZ2_7TeV-madgraph-tauola/Spring...,169649,74,0.2,0.0,0.0,3790,WJetsToLNu_TuneZ2_7TeV-madgraph-tauola,Spring11-PU_S1_START311_V1G1-v1,GEN-SIM-RECO
25206,/WJetsToLNu_TuneZ2_7TeV-madgraph-tauola/dstrom...,124621,69,0.1,0.0,0,0,WJetsToLNu_TuneZ2_7TeV-madgraph-tauola,dstrom-prod_2011_10_05_17_14_11-bf57a985b107a6...,USER
18527,/WJetsToLNu_TuneZ2_7TeV-madgraph-tauola/mschro...,85831,64,0.1,0.0,0,0,WJetsToLNu_TuneZ2_7TeV-madgraph-tauola,mschrode-RA2PreSelectionOnMC_Summer11-PU_S4_ST...,USER
22673,/WJetsToLNu_TuneZ2_7TeV-madgraph-tauola/lhx-WJ...,33616,62,0.0,0.0,0,0,WJetsToLNu_TuneZ2_7TeV-madgraph-tauola,lhx-WJetsToLNu_TuneZ2-madgraph_Summer11_ra2Std...,USER
145,/WJetsToLNu_TuneZ2_7TeV-madgraph-tauola/Fall11...,89020,52,0.1,0.0,0.1,80508,WJetsToLNu_TuneZ2_7TeV-madgraph-tauola,Fall11-PU_S6_START44_V9B-v1,AODSIM


Build a new dataframe with sums by process

In [28]:
mc_df['NUSERS'].loc[mc_df['process'] == "WJetsToLNu_TuneZ2_7TeV-madgraph-tauola"]

0        3667
5        1180
27       1076
1449      111
138       105
887        74
25206      69
18527      64
22673      62
145        52
24407      39
25188      36
21832      35
18354      35
17613      31
1402       29
3374       28
1686       26
18359      26
25225      25
2311       23
19258      22
1241       20
448        20
3295       20
24261      18
1276       17
22455      16
19139      16
1697       14
         ... 
19796       2
6228        2
20063       2
24902       2
9842        2
12804       2
12864       2
18966       2
9303        2
20981       1
23932       1
23919       1
24752       1
26361       1
26251       1
22983       1
13225       1
5459        1
8464        1
8549        1
8223        1
6736        1
9336        1
17963       1
2997        1
2923        1
2670        1
2686        1
15422       1
14570       1
Name: NUSERS, dtype: int64

In [29]:
sum(mc_df['NUSERS'].loc[mc_df['process'] == "WJetsToLNu_TuneZ2_7TeV-madgraph-tauola"])

7496

In [30]:
sum(mc_df['NUSERS'].loc[mc_df['process'] == mc_df['process'][0]])

7496

In [31]:
df_byprocess = pd.DataFrame({'process': mc_df['process']})

In [32]:
df_byprocess.head(6)

,process
1,DYJetsToLL_TuneZ2_M-50_7TeV-madgraph-tauola
0,WJetsToLNu_TuneZ2_7TeV-madgraph-tauola
23,TTJets_TuneZ2_7TeV-madgraph-tauola
6,DYJetsToLL_TuneZ2_M-50_7TeV-madgraph-tauola
5,WJetsToLNu_TuneZ2_7TeV-madgraph-tauola
27,WJetsToLNu_TuneZ2_7TeV-madgraph-tauola


Remove multiple entries

In [33]:
df_byprocess = df_byprocess.drop_duplicates()

Add summed value for NUSERS and NACC to the new dataframe

In [34]:
sum(mc_df['NUSERS'].loc[mc_df['process'] == df_byprocess['process'][0]])

7496

Sum the values of NUSERS and NACC for each original entry. This is very slow, there must be a faster way of doing it.

In [35]:
nusers = []
nacc = []
for index, row in df_byprocess.iterrows():

# Sum only those positions (loc) where the process name in the full list is the same as the process name in the process list
    nusers.append(sum(mc_df['NUSERS'].loc[mc_df['process'] == df_byprocess['process'][index]]))
    nacc.append(sum(mc_df['NACC'].loc[mc_df['process'] == df_byprocess['process'][index]]))



Add the new summed values to the process dataframe

In [36]:
df_byprocess['nusers'] = nusers
df_byprocess['nacc'] = nacc

df_byprocess.head(6)    



,process,nusers,nacc
1,DYJetsToLL_TuneZ2_M-50_7TeV-madgraph-tauola,7048,4020412
0,WJetsToLNu_TuneZ2_7TeV-madgraph-tauola,7496,6120395
23,TTJets_TuneZ2_7TeV-madgraph-tauola,6691,2643341
10,QCD_Pt-30to80_EMEnriched_TuneZ2_7TeV-pythia,1177,1274263
101,T_TuneZ2_t-channel_7TeV-powheg-tauola,1194,194141
46,DYToMuMu_M-20_CT10_TuneZ2_7TeV-powheg-pythia,2022,1244117


Order the new list by number of users

In [37]:
df_byprocess.sort_values('nusers',ascending=False)

,process,nusers,nacc
0,WJetsToLNu_TuneZ2_7TeV-madgraph-tauola,7496,6120395
1,DYJetsToLL_TuneZ2_M-50_7TeV-madgraph-tauola,7048,4020412
23,TTJets_TuneZ2_7TeV-madgraph-tauola,6691,2643341
34,QCD_Pt-20_MuEnrichedPt-15_TuneZ2_7TeV-pythia6,2121,1128798
46,DYToMuMu_M-20_CT10_TuneZ2_7TeV-powheg-pythia,2022,1244117
468,Hydjet_Bass_MinBias_2760GeV,1532,577072
349,DYToMuMu_M-20_TuneZ2_7TeV-pythia6,1442,334359
21,QCD_Pt-20to30_EMEnriched_TuneZ2_7TeV-pythia6,1420,1093268
340,DYToEE_M-20_TuneZ2_7TeV-pythia6,1244,364423
79,DYToTauTau_M-20_CT10_TuneZ2_7TeV-powheg-pythia...,1244,491433


In [38]:
new_df.head(100)

,COLLNAME,NACC,NUSERS,RNACC,RNUSERS,RTOTCPU,TOTCPU,process,reprocessing,datatype
1,/DYJetsToLL_TuneZ2_M-50_7TeV-madgraph-tauola/S...,2026464,3786,1.9,1.3,3.3,2056789,DYJetsToLL_TuneZ2_M-50_7TeV-madgraph-tauola,Summer11-PU_S4_START42_V11-v1,AODSIM
0,/WJetsToLNu_TuneZ2_7TeV-madgraph-tauola/Summer...,2900135,3667,2.7,1.3,5.2,3296253,WJetsToLNu_TuneZ2_7TeV-madgraph-tauola,Summer11-PU_S4_START42_V11-v1,AODSIM
4,/SingleMu/Run2011A-PromptReco-v4/AOD,1537482,3571,1.4,1.3,2.8,1765946,SingleMu,Run2011A-PromptReco-v4,AOD
7,/SingleMu/Run2011A-May10ReReco-v1/AOD,2851106,3082,2.6,1.1,1.5,966514,SingleMu,Run2011A-May10ReReco-v1,AOD
3,/SingleMu/Run2011B-PromptReco-v1/AOD,1262370,2238,1.2,0.8,3.0,1915483,SingleMu,Run2011B-PromptReco-v1,AOD
18,/DoubleMu/Run2011A-PromptReco-v4/AOD,502407,1959,0.5,0.7,0.8,511015,DoubleMu,Run2011A-PromptReco-v4,AOD
11,/DoubleElectron/Run2011A-PromptReco-v4/AOD,657485,1942,0.6,0.7,1.2,746178,DoubleElectron,Run2011A-PromptReco-v4,AOD
23,/TTJets_TuneZ2_7TeV-madgraph-tauola/Summer11-P...,329509,1898,0.3,0.7,0.7,434610,TTJets_TuneZ2_7TeV-madgraph-tauola,Summer11-PU_S4_START42_V11-v1,AODSIM
40,/DoubleMu/Run2011A-May10ReReco-v1/AOD,680902,1739,0.6,0.6,0.5,312152,DoubleMu,Run2011A-May10ReReco-v1,AOD
32,/DoubleElectron/Run2011A-May10ReReco-v1/AOD,891548,1571,0.8,0.6,0.6,351559,DoubleElectron,Run2011A-May10ReReco-v1,AOD
